## RNN with Adaptive Softmax

[Efficient softmax approximation for GPUs](https://arxiv.org/pdf/1609.04309v3.pdf)


In [1]:
%load_ext autoreload
%autoreload 2
import torch
import torchtext
from torchtext import data

TEXT = data.Field(
    tokenizer_language='en',
    lower=True,
    init_token='<sos>',
    eos_token='<eos>',
    batch_first=True,
)

Now, we can load our dataset

In [2]:
from torchtext.datasets import WikiText2
 
train, valid, test = WikiText2.splits(TEXT) 

TEXT.build_vocab(train, vectors="glove.6B.300d", min_freq=5)

print(f"We have {len(TEXT.vocab)} tokens in our vocabulary")

We have 20490 tokens in our vocabulary


## Iterator


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

BATCH_SIZE = 32
BPTT_LEN = 35

train_iter, valid_iter, test_iter = data.BPTTIterator.splits(
    (train, valid, test),
    batch_size=BATCH_SIZE,
    bptt_len=BPTT_LEN, # this is where we specify the sequence length
    device=device,
    repeat=False)

In [4]:
import warnings
warnings.filterwarnings('ignore')
from torch import nn
import torch

seq_len, batch_size, hidden_size = 7, 20, 256
size = (seq_len, batch_size, hidden_size)
vocab_size = len(TEXT.vocab)

X = torch.randn(seq_len, batch_size, hidden_size)

trg = torch.randint(0, vocab_size, (seq_len * batch_size,))

out = nn.AdaptiveLogSoftmaxWithLoss(
    hidden_size, vocab_size, cutoffs=[10, 100, 1000]
)

X = X.view(-1, X.shape[-1])

print(X.shape, trg.shape)
asm_out = out(X, trg)

asm_out.loss

torch.Size([140, 256]) torch.Size([140])


In [6]:
# %load ../pytorch_lm/models/qrnn.py
import torch.nn as nn
from torchqrnn import QRNN


class RNNAdapLanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, pad_idx, hidden_size,
                 cell_class=nn.GRU, dropout=0.20, zoneout=.0):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=PAD_IDX)

        #self.qrnn = QRNN(embedding_dim, hidden_size, num_layers=2, window=2, dropout=dropout, zoneout=zoneout)
        self.rnn = cell_class(embedding_dim, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.out = nn.AdaptiveLogSoftmaxWithLoss(
            hidden_size, vocab_size, 
            cutoffs=[500, 2000, 10000],
        )

    def forward(self, inp, target=None, hidden=None):
        """
        Inputs are supposed to be just one step (i.e. one letter)
        """

        # inputs = [batch_size, seqlen]
        emb = self.embedding(inp)
        # emb = [batch, seqlen, embedding_dim]
        outputs, hidden = self.rnn(emb, hidden)
        # outputs = [batch, seqlen, hidden_size]
        outputs = outputs.contiguous().view(-1, outputs.shape[-1])
        # outputs = [seqlen * batch, hidden_size]
        outputs = self.dropout(outputs)
        asm_out = self.out(outputs, target)

        return asm_out, hidden


Create the Language Model

In [12]:
# %load ../pytorch_lm/training.py
from tqdm.auto import tqdm
import torch
import math
import numpy as np


def display_lr(lr):
    pow = math.floor(math.log10(lr))
    return f"{lr*(10**(-pow)):.2f}e{pow}"

def train(model, iterator, optimizer, criterion, clip_norm=None, ncols=None):
    """
    Trains the model for one full epoch
    """
    epoch_loss = 0
    epoch_perplexity = 0

    model.train()

    epoch_bar = tqdm(iterator, total=len(iterator), ncols=ncols)

    i = 0
    for batch in epoch_bar:
        i += 1
        optimizer.zero_grad()
        text = batch.text
        trg = batch.target.view(-1)

        out, _ = model(text, target=trg)
        if type(out) is torch.Tensor:
            # Not adaptive softmax
            # Use criterion
            preds = out.view(-1, out.shape[-1])
            loss = criterion(preds, trg)
        else:
            # Adaptive softmax
            loss = out.loss
        
        loss.backward()

        total_norm = 0

        for p in model.parameters():
            param_norm = p.grad.data.norm(2)
            total_norm += param_norm.item() ** 2

        total_norm = total_norm ** (1. / 2)

        if clip_norm:
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_norm)

        optimizer.step()

        epoch_loss += loss.item()
        epoch_perplexity += np.exp(loss.item())

        lr = optimizer.param_groups[0]["lr"]

        epoch_bar.set_description(f"norm = {total_norm:.5f} loss = {epoch_loss / i:.4f} LR = {display_lr(lr)}")

    return epoch_loss / len(iterator), epoch_perplexity / len(iterator)


def evaluate(model, iterator, criterion=None):
    """
    Evaluates the model on the given iterator
    """
    epoch_loss = .0
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            text = batch.text
            trg = batch.target.view(-1)

            out, _ = model(text, target=trg)
            if type(out) is torch.Tensor:
                # Not adaptive softmax
                # Use criterion
                preds = out.view(-1, out.shape[-1])
                loss = criterion(preds, trg)
            else:
                # Adaptive softmax
                loss = out.loss
            
            epoch_loss += loss.item()

        loss = epoch_loss / len(iterator)

        perplexity = np.exp(loss)

    return loss, perplexity

def training_cycle(model, train_iter, valid_iter, epochs,
                   optimizer, criterion, scheduler, model_path,
                   early_stopping_tolerance=None, ncols=None):

    best_valid_loss = float('inf')
    epochs_without_improvement = 0

    for epoch in range(epochs):
        print(f"Epoch {epoch+1}")

        train_loss, train_perplexity = train(model, train_iter, optimizer, criterion, ncols=ncols)
        valid_loss, valid_perplexity = evaluate(model, valid_iter, criterion)

        scheduler.step(valid_loss)

        desc = f' Train Loss: {train_loss:.5f} Perp: {train_perplexity:.3f}'
        desc += f' Val. Loss: {valid_loss:.5f} Perp: {valid_perplexity:.3f}'

        print(desc)
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            epochs_without_improvement = 0
            torch.save(model.state_dict(), model_path)
            print(f"Best model so far (Loss {best_valid_loss:.5f} Perp {valid_perplexity:.2f}) saved at {model_path}")
        else:
            epochs_without_improvement += 1
            if early_stopping_tolerance and epochs_without_improvement >= early_stopping_tolerance:
                print("Early stopping")
                break


## Training 

In [8]:
import torch.optim as optim


PAD_IDX = TEXT.vocab.stoi["<pad>"]
UNK_IDX = TEXT.vocab.stoi["<unk>"]


HIDDEN_DIM = 512
vocab_size = TEXT.vocab.vectors.shape[0]
embedding_dim = TEXT.vocab.vectors.shape[1]

model_path = "/tmp/qrnn_lang_model.pt"

model = QRNNAdaSoftmaxLanguageModel(
    vocab_size, embedding_dim, dropout=0.1, zoneout=0.1,
    hidden_size=HIDDEN_DIM, pad_idx=PAD_IDX)

# Set weight for UNK to a random normal
model.embedding.weight.data.copy_(TEXT.vocab.vectors)
model.embedding.weight.data[UNK_IDX] = torch.randn(embedding_dim)


optimizer = optim.SGD(model.parameters(), lr=1)
criterion = nn.CrossEntropyLoss()

lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=4, threshold=0.0001)

model = model.to(device)
criterion = criterion.to(device)



In [9]:

N_EPOCHS = 400

best_valid_loss = float('inf')

early_stopping_tolerance = 39



training_cycle(
    epochs=200, ncols=800,
    model=model, train_iter=train_iter, valid_iter=valid_iter, 
    optimizer=optimizer, criterion=criterion, scheduler=lr_scheduler,
    model_path=model_path, early_stopping_tolerance=7
)

Epoch 1



 Train Loss: 6.32825 Perp: 6919978294719.197 Val. Loss: 5.60865 Perp: 272.775
Best model so far (Loss 5.60865 Perp 272.78) saved at /tmp/qrnn_lang_model.pt
Epoch 2



 Train Loss: 5.56768 Perp: 264.571 Val. Loss: 5.35646 Perp: 211.972
Best model so far (Loss 5.35646 Perp 211.97) saved at /tmp/qrnn_lang_model.pt
Epoch 3



 Train Loss: 5.31514 Perp: 205.277 Val. Loss: 5.20139 Perp: 181.525
Best model so far (Loss 5.20139 Perp 181.52) saved at /tmp/qrnn_lang_model.pt
Epoch 4



 Train Loss: 5.13625 Perp: 171.551 Val. Loss: 5.09949 Perp: 163.938
Best model so far (Loss 5.09949 Perp 163.94) saved at /tmp/qrnn_lang_model.pt
Epoch 5


...


 Train Loss: 4.22853 Perp: 69.260 Val. Loss: 4.83860 Perp: 126.292
Epoch 65



 Train Loss: 4.22849 Perp: 69.252 Val. Loss: 4.83860 Perp: 126.292
Epoch 66



 Train Loss: 4.22904 Perp: 69.294 Val. Loss: 4.83860 Perp: 126.292
Epoch 67



 Train Loss: 4.22859 Perp: 69.261 Val. Loss: 4.83860 Perp: 126.292
Epoch 68



 Train Loss: 4.22896 Perp: 69.295 Val. Loss: 4.83860 Perp: 126.292
Early stopping


In [13]:


model.load_state_dict(torch.load(model_path))
model.eval()

valid_loss, valid_perplexity = evaluate(model, valid_iter, criterion)
test_loss, test_perplexity = evaluate(model, test_iter, criterion)


print(f"Valid loss      : {valid_loss:.2f}")
print(f"Valid perplexity: {valid_perplexity:.2f}\n")

print(f"Test loss      : {test_loss:.2f}")
print(f"Test perplexity: {test_perplexity:.2f}")

Valid loss      : 4.84
Valid perplexity: 126.29

Test loss      : 4.78
Test perplexity: 119.49


We can check perplexities for other models in [this blogpost](https://blog.einstein.ai/the-wikitext-long-term-dependency-language-modeling-dataset/)

A more complex recurrent network (using a cache of hidden states) achieves a perplexity of 100. So this very basic model (without any hyperparameter optimization) seems fairly ok

## Sampling

In [ ]:
import numpy as np
import torch.nn.functional as F

def sample_sentence(init_token="<eos>", temperature=1):

    seq = [TEXT.vocab.stoi[init_token]]
    hidden = None
    while len(seq) == 1 or seq[-1] != EOS_IDX:
        inp = torch.LongTensor([[seq[-1]]]).to(device)
        out, hidden = model(inp, hidden=hidden)

        """
        Sample from probabilities
        """
        probs = F.softmax(out.view(-1) / temperature, dim=0)
        next_tok_idx = torch.multinomial(probs, num_samples=1)
        
        seq.append(next_tok_idx)
        
    return [TEXT.vocab.itos[t] for t in seq]

In [ ]:
for temperature in np.arange(0.5, 2.5, 0.10):
    print("="*80, f"\nSampling with temperature = {temperature:.2f}")
    
    print(" ".join(sample_sentence("the", temperature=temperature)))

We can observe that:

- with hidden states there are more "meaningful" stuff
- quotation marks are closed when using the hidden state